### Preprocess한 Maestro dataset .npy 파일들을 다운로드 합니다.

In [9]:
# Google drive에서 받기 위해 gdown을 설치합니다
!pip install gdown
# maestro-npy.zip 파일을 다운로드 합니다.
!gdown https://drive.google.com/uc?id=1PCsy4C0XdO2hWedw7w6FZ39neCgAdNAi
# maestro-npy.zip 압축파일을 풉니다.
!unzip -o -qq maestro-npy.zip

Downloading...
From: https://drive.google.com/uc?id=1PCsy4C0XdO2hWedw7w6FZ39neCgAdNAi
To: /home/gaudio/ste/projects/lectures/AI-for-Music/maestro-npy.zip
100%|████████████████████████████████████████| 124M/124M [00:28<00:00, 4.29MB/s]


In [22]:
import os
import numpy as np
from torch.utils.data import Dataset, DataLoader

### Dataset에 관련된 hyper-parameters를 정의합니다.

In [23]:
from easydict import EasyDict
# 각 이벤트 종류 (interval, velocity 등)에 대해 사용할 token의 갯수를 지정합니다.
dims = EasyDict(interval = 100,
                velocity = 32,
                note_on = 128,
                note_off = 128,
                pedal_on = 1,
                pedal_off = 1)

# 각 이벤트 종류 (interval, velocity 등)에 대해 token의 offset을 지정합니다.
offsets = EasyDict(interval = 100,
                   velocity = dims.interval,
                   note_on = dims.interval + dims.velocity,
                   note_off = dims.interval + dims.velocity + dims.note_on,
                   pedal_on = dims.interval + dims.velocity + dims.note_on + dims.note_off,
                   pedal_off = dims.interval + dims.velocity + dims.note_on + dims.note_off + dims.pedal_on)

# Dataset에 사용될 hyper-parameters를 지정합니다.
dataset_hparams = EasyDict(root_dir = 'dataset/',
                           max_note_duration = 2, # seconds
                           token_length = 1024,
                           dims = dims,
                           offsets = offsets
                          )
for hp in dataset_hparams:
    print(hp, dataset_hparams[hp])

root_dir dataset/
max_note_duration 2
token_length 1024
dims {'interval': 100, 'velocity': 32, 'note_on': 128, 'note_off': 128, 'pedal_on': 1, 'pedal_off': 1}
offsets {'interval': 100, 'velocity': 100, 'note_on': 132, 'note_off': 260, 'pedal_on': 388, 'pedal_off': 389}


### event_list를 token sequence로 변환하는 함수를 만듭니다.

In [24]:
def event_list_to_tokens(event_list, hp):
    # token들을 모아줄 list를 만듭니다.
    tokens = []
    prev_time = 0
    
    # event_list에서 event들을 하나씩 꺼내옵니다.
    for event in event_list:
        
        # interval은 현재 event['time']에서 prev_time을 빼서 구합니다.
        interval = event['time'] - prev_time
        
        # interval(시간)을 interval_token(자연수) 값으로 변환합니다.
        interval_token = int(interval / hp.max_note_duration * hp.dims.interval)
        interval_token = min(interval_token, hp.dims.interval)
        
        # tokens에 interval_token을 저장합니다.
        tokens.append(interval_token)
        
        # 현재 event['time']을 prev_time으로 저장합니다.
        prev_time = event['time']
        
        # 이벤트 'note_on'의 경우 velocity token과 note_on token을 저장합니다.
        if event['type'] == 'note_on':
            tokens.append(hp.offsets.velocity + int(event['velocity'] / 128 * hp.dims.velocity))
            tokens.append(hp.offsets.note_on + event['note'])
            
        # 이벤트 'note_off'에 대한 token을 저장합니다.
        elif event['type'] == 'note_off':
            tokens.append(hp.offsets.note_off + event['note'])
            
        # 이벤트 'pedal_on'에 대한 token을 저장합니다.
        elif event['type'] == 'pedal_on':
            tokens.append(hp.offsets.pedal_on)
            
        # 이벤트 'pedal_off'에 대한 token을 저장합니다.
        elif event['type'] == 'pedal_off':
            tokens.append(hp.offsets.pedal_off)
            
    # token sequence를 반환합니다.
    return np.array(tokens)


### Pytorch framework에서 사용하는 규격대로 Dataset class를 정의합니다.

In [25]:
class MaestroDataset(Dataset):
    def __init__(self, dataset_hparams):
        super().__init__()
        self.hp = dataset_hparams
        # .npy 파일의 목록을 저장합니다.
        self.files = [os.path.join(self.hp.root_dir, file) for file in os.listdir(self.hp.root_dir) if 'npy' in file]
    
    # 전체 data 갯수를 반환합니다.
    def __len__(self):
        return len(self.files)
    
    # 하나의 data index를 받아 token sequence를 반환합니다.
    def __getitem__(self, index):
        # file 경로를 구합니다.
        file = self.files[index]
        
        # .npy 파일을 읽어 event_list를 만듭니다.
        event_list = np.load(file, allow_pickle=True)
        
        # event_list를 token sequence로 변환합니다.
        tokens = event_list_to_tokens(event_list, self.hp)
        
        # dataset에 사용될 만큼 token sequence를 자릅니다.
        if len(tokens) < self.hp.token_length:
            start_index = 0
        else:
            start_index = np.random.randint(0, len(tokens)-self.hp.token_length)
        tokens = np.array(tokens[start_index:start_index+self.hp.token_length])
        
        # token sequence가 dataset에 사용될 길이보다 짧은 경우 padding을 합니다.
        tokens_padded = np.zeros(self.hp.token_length, dtype=np.int)
        tokens_padded[:len(tokens)] = tokens
        
        # padding된 token sequence를 반환합니다.
        return np.array(tokens_padded)

### Pytorch에서 사용할 수 있도록 dataset과 train_loader를 만듭니다.

In [26]:
dataset = MaestroDataset(dataset_hparams)
print(dataset)

train_loader = DataLoader(dataset, batch_size=16)
print(train_loader)

In [30]:
for batch in train_loader:
    print(batch.shape)
    for i, data in enumerate(batch):
        print(str(i) + '번 token sequence :', data.numpy())
    break

torch.Size([16, 1024])
0번 token sequence : [  0 111 175 ... 208   2 336]
1번 token sequence : [119 202   1 ...   1 321   0]
2번 token sequence : [118 194   3 ...   0 118 200]
3번 token sequence : [308   1 349 ...   0 117 182]
4번 token sequence : [  0 336   0 ...   0 124 187]
5번 token sequence : [  0 117 216 ...   1 120 222]
6번 token sequence : [114 191   0 ...   0 116 185]
7번 token sequence : [179   5 307 ... 343   7 323]
8번 token sequence : [  0 339   3 ... 121 219   0]
9번 token sequence : [116 206   0 ... 200   0 123]
10번 token sequence : [194   3 322 ...   3 328   3]
11번 token sequence : [117 201   2 ...   6 117 197]
12번 token sequence : [184   1 324 ...   2 119 204]
13번 token sequence : [338   1 331 ... 388   1 312]
14번 token sequence : [197   1 322 ... 117 198   0]
15번 token sequence : [115 204   0 ... 116 200   3]
